In [3]:
using JSON,Knet
include(Pkg.dir("Knet","data","imagenet.jl"));
include(Pkg.dir("Knet","examples","vgg","vgg.jl"));

In [8]:
softmax(w,data,predfun;actfun=sigm) = mean(softmax(w,x,y,predfun,actfun) for (x,y) in data)
softmax(w,x,y,predfun; actfun=sigm, o...) = nll(predfun(w,x,actfun;o...),y)
softgrad = grad(softmax)
zeroone(w,data,model) = 1 - accuracy(w,data,model)

34-element Array{Any,1}:
 Any[Any[Float32[131.32 131.32 … 131.32 131.32; 131.32 131.32 … 131.32 131.32; … ; 131.32 131.32 … 131.32 131.32; 131.32 131.32 … 131.32 131.32]

Float32[134.221 135.221 … 138.221 138.221; 135.221 137.221 … 138.221 138.221; … ; 138.221 138.221 … 134.221 134.221; 138.221 138.221 … 134.221 132.221]

Float32[141.061 144.061 … 151.061 151.061; 144.061 148.061 … 151.061 151.061; … ; 151.061 151.061 … 151.061 151.061; 151.061 151.061 … 151.061 151.061], 8], Any[Float32[131.32 131.32 … 131.32 131.32; 131.32 131.32 … 131.32 131.32; … ; 131.32 131.32 … 131.32 131.32; 131.32 131.32 … 131.32 131.32]

Float32[138.221 138.221 … 138.221 138.221; 138.221 138.221 … 138.221 138.221; … ; 138.221 138.221 … 138.221 138.221; 138.221 138.221 … 138.221 138.221]

Float32[151.061 151.061 … 151.061 151.061; 151.061 151.061 … 151.061 151.061; … ; 151.061 151.061 … 151.061 151.061; 151.061 151.061 … 151.061 151.061], 2], Any[Float32[96.32 98.32 … 96.32 103.32; 98.32 98.32 … 67.32 75.32; …

In [4]:
function minibatch(data,batchsize)
    rng = MersenneTwister(1234);
    shuffled=shuffle!(rng, data)
    batches = []
    for k = 1:batchsize:length(shuffled)
        lo = k
        hi = min(k+batchsize-1, length(shuffled))
        samples = shuffled[lo:hi]
        push!(batches,samples)
    end
    return batches
end
    

minibatch (generic function with 1 method)

In [19]:
#generates [x,y] pairs where x=[224,224,3,1] and y=1:classnumber
function prepareData(model,filenames,atype)
    badurl=0
    _y=Any[]#class label
    vgg = Main.matconvnet(model) #vgg = Main.matconvnet(o[:model])
    averageImage = convert(Array{Float32},vgg["meta"]["normalization"]["averageImage"])
    imgUrls=Any[]
    #open file and read the data push 
    for class in 1:length(filenames)
        open(string("Data/"filenames[class],".json"),"r") do f
            jsonObj=Any[] #global
            lines=readlines(f)
            for l in lines
                dict2=JSON.parse(l)
                push!(jsonObj,dict2)
                push!(_y,class) #extract the class information
                push!(imgUrls,get(dict2, "imUrl", nothing)) #extract the imgurls from jason
            end
        end
    end
    
   x=Any[]
    data=Any[]
   clean_y=Any[]
    for i in 1:length(imgUrls)
        if(imgUrls[i]!=nothing)
        #println(url)
            try
            image = Main.imgdata(imgUrls[i], averageImage);
            image = convert(atype, image);
            push!(x, image)
            push!(clean_y,_y[i])
            push!(data, [image,_y[i]])
            catch
                badurl+=1
                #println(imgUrls[i])
                println("url is not good")
            end
        else
            badurl+=1
        end
        
    end
    #clean_y=convert(atype,clean_y)
    return (imgUrls,data)
end
            

prepareData (generic function with 1 method)

In [64]:
    atype=KnetArray{Float32}
    _filenames=["meta_Baby", "meta_Cell_Phones_and_Accessories","meta_Clothing_Shoes_and_Jewelry",
          "meta_Electronics","meta_Grocery_and_Gourmet_Food","meta_Health_and_Personal_Care",
          "meta_Home_and_Kitchen","meta_Musical_Instruments","meta_Pet_Supplies",
          "meta_Sports_and_Outdoors","meta_Tools_and_Home_Improvement","meta_Toys_and_Games"]
    model="imagenet-vgg-verydeep-16" 
    (imgUrls,data)=prepareData(model,_filenames,atype);
    vgg = Main.matconvnet(model)
    params = VGG.get_params(vgg, atype)
    convnet = VGG.get_convnet(params...)
    @time y1 = convnet(data[1][1])
    z1 = vec(Array(y1))
    s1 = sortperm(z1,rev=true)
    description = vgg["meta"]["classes"]["description"]
    p1 = exp.(logp(z1))
    display(hcat(p1[s1[1:5]], description[s1[1:5]]))

5×2 Array{Any,2}:
 0.843046   "bulletproof vest"                                            
 0.0759644  "knee pad"                                                    
 0.0248794  "sweatshirt"                                                  
 0.0109966  "mailbag, postbag"                                            
 0.0105968  "backpack, back pack, knapsack, packsack, rucksack, haversack"

  0.035217 seconds (1.72 k allocations: 85.906 KiB)


In [68]:
p1[s1[1:5]]

5-element Array{Float32,1}:
 0.843046 
 0.0759644
 0.0248794
 0.0109966
 0.0105968

In [51]:
length(vgg["layers"])

37

LoadError: [91mBoundsError: attempt to access 1×2 Array{Any,2} at index [Base.Slice(Base.OneTo(1)), Base.Slice(Base.OneTo(2)), Base.Slice(Base.OneTo(1)), 1:12][39m